In [1]:
import torch
from dataset import DogHeartTestDataset
from torch.utils.data import DataLoader
from model import get_model
from utils import get_transform
from evaluate import plot_predictions
import os

current_dir = os.getcwd()

# Navigate up three levels to reach the project root
project_root = os.path.abspath(os.path.join(current_dir, '..'))

# Create the path to static/uploads
uploads_dir = os.path.join(project_root, 'project','static', 'uploads','Images')
uploads_dir


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = get_model(device)
checkpoint_path = 'final_best_model.pth' # Download at https://drive.google.com/file/d/1-nng3RxjHvUl5O6q8D8xYGoXXRdGpgKh/view?usp=drive_link
checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=True)
model.load_state_dict(checkpoint)

test_dataset = DogHeartTestDataset(uploads_dir, transforms=get_transform(512))
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

#plot_predictions(model, test_loader, device) # Due to GitHub's file upload size restrictions, only a subset of the images is displayed below

In [5]:
import os
import torch
import scipy.io as sio
import numpy as np
from PIL import Image

PREDICT_FOLDER = "static/predict"  # Directory to save .mat files
os.makedirs(PREDICT_FOLDER, exist_ok=True)  # Ensure folder exists


In [7]:
## Save the Predictions to .mat files
def calc_vhs(x: torch.Tensor):
    """Compute VHS value based on six predicted points."""
    A, B = x[..., 0:2], x[..., 2:4]
    C, D = x[..., 4:6], x[..., 6:8]
    E, F = x[..., 8:10], x[..., 10:12]

    AB = torch.norm(A - B, p=2, dim=-1)
    CD = torch.norm(C - D, p=2, dim=-1)
    EF = torch.norm(E - F, p=2, dim=-1)

    vhs = 6 * (AB + CD) / EF
    return vhs

def save_predictions_to_mat(image_name, predicted_points, vhs_value):
    """Save the predicted points and VHS value into a .mat file."""
    mat_data = {
        "six_points": predicted_points.numpy(),  # Store predicted six points
        "VHS": np.array([[vhs_value]])  # Store VHS as a single-element array
    }

    mat_filename = os.path.join(PREDICT_FOLDER, f"{os.path.splitext(image_name)[0]}.mat")
    sio.savemat(mat_filename, mat_data)
    print(f"✅ Predictions saved to {mat_filename}")


In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = get_model(device)
checkpoint_path = 'final_best_model.pth' # Download at https://drive.google.com/file/d/1-nng3RxjHvUl5O6q8D8xYGoXXRdGpgKh/view?usp=drive_link
checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=True)
model.load_state_dict(checkpoint)


<All keys matched successfully>

In [21]:
test_dataset = DogHeartTestDataset(uploads_dir, transforms=get_transform(512))
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

print(f"Number of batches in the test DataLoader: {len(test_loader)}")

Number of batches in the test DataLoader: 2


In [26]:
import matplotlib.pyplot as plt
import scipy.io as sio


def save_predictions_to_mat(image_name, predicted_points, vhs_value):
    """Save predicted points and VHS in MATLAB-compatible .mat format."""
    mat_filename = os.path.splitext(image_name)[0] + ".mat"
    mat_filepath = os.path.join(PREDICT_FOLDER, mat_filename)

    # Convert points to correct MATLAB format
    mat_data = {
        "six_points": np.array(predicted_points),  # Store 6 points in an array
        "VHS": np.array([[vhs_value]])  # Store VHS as a 2D array
    }

    sio.savemat(mat_filepath, mat_data)
    print(f"✅ Predictions saved in MATLAB format: {mat_filepath}")


# Function to plot images with their predicted and true points (if applicable).
def generate_predictions(model, data_loader, device):
    """
    Plot predictions and print point coordinates for each image.
    
    - True points are shown in blue.
    - Predicted points are shown in red.
    - VHS values (True vs Predicted) are displayed.
    - X1, Y1, X2, Y2 coordinates for all 6 points are printed.
    """
    model.eval()  
    with torch.no_grad():
        for data_batch in data_loader:
            if len(data_batch) == 4:
                image_names, images, points, vhs_gt = data_batch
            elif len(data_batch) == 2:
                images, image_names = data_batch
                points, vhs_gt = None, None  # No true points for unlabeled images
            else:
                print("Unexpected batch format")
                continue
            
            images = images.to(device)
            outputs = model(images)  # Generate predictions
            vhs_pred = calc_vhs(outputs)  # Compute VHS values
            
            # Convert tensors to numpy
            images_np = images.permute(0, 2, 3, 1).cpu().numpy()
            outputs_np = outputs.cpu().numpy()
            vhs_pred_np = vhs_pred.cpu().numpy()
            
            ## start : Loop through image names

            for i,name in enumerate(image_names):
                print(i,name)
                img = images_np[i]
                img = (img - img.min()) / (img.max() - img.min())  # Normalize for display
                pred_points = outputs_np[i].reshape(-1, 2) * img.shape[0]
                print("pred points=>",len(pred_points))
                print(pred_points)
                pred_vhs = vhs_pred_np[i].item()
                print("vhs=>",pred_vhs)

                 # Save predictions in MATLAB format
                save_predictions_to_mat(name, pred_points, pred_vhs)
                

            ## end : Loop through image names

generate_predictions(model,test_loader,device)

0 10_12.8_1.png
pred points=> 6
[[271.37186 235.97198]
 [343.1939  476.76062]
 [174.53072 355.34903]
 [356.82278 275.9262 ]
 [198.82953 231.92859]
 [404.1997  164.09407]]
vhs=> 12.486807823181152
✅ Predictions saved in MATLAB format: static/predict/10_12.8_1.mat
1 1_12.3_1.png
pred points=> 6
[[244.76201 210.54951]
 [284.64145 447.82596]
 [150.48428 322.71024]
 [320.647   281.01227]
 [152.877   140.29593]
 [360.2752  138.80406]]
vhs=> 12.028769493103027
✅ Predictions saved in MATLAB format: static/predict/1_12.3_1.mat
2 2_10.8_0.png
pred points=> 6
[[246.73949 234.6333 ]
 [306.78677 415.8583 ]
 [195.3375  308.02878]
 [296.6527  267.6255 ]
 [212.03813 157.40602]
 [371.07993 117.36887]]
vhs=> 10.97492504119873
✅ Predictions saved in MATLAB format: static/predict/2_10.8_0.mat
3 3_11.6_1.png
pred points=> 6
[[255.53868 195.21126]
 [351.28363 423.42432]
 [181.34215 332.7177 ]
 [327.12778 227.59262]
 [194.02226 129.62285]
 [397.69598  70.89267]]
vhs=> 12.092692375183105
✅ Predictions saved i

In [2]:
import scipy

In [28]:
mat_data = scipy.io.loadmat("static/predict/5_10.3_0.mat")
mat_data

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Tue Feb  4 23:24:00 2025',
 '__version__': '1.0',
 '__globals__': [],
 'six_points': array([[299.52444 , 160.72844 ],
        [397.1723  , 452.7952  ],
        [233.1101  , 332.27368 ],
        [387.5871  , 211.44118 ],
        [163.38231 , 143.90636 ],
        [417.69897 ,  33.748253]], dtype=float32),
 'VHS': array([[10.9128027]])}

In [1]:
import pandas as pd
import csv

img_size = 512

def calc_vhs(x: torch.Tensor):
    """Compute VHS value based on six predicted points."""
    A, B = x[..., 0:2], x[..., 2:4]
    C, D = x[..., 4:6], x[..., 6:8]
    E, F = x[..., 8:10], x[..., 10:12]

    AB = torch.norm(A - B, p=2, dim=-1)
    CD = torch.norm(C - D, p=2, dim=-1)
    EF = torch.norm(E - F, p=2, dim=-1)

    vhs = 6 * (AB + CD) / EF
    return vhs


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = get_model(device)
#checkpoint_path = 'final_best_model.pth' # Download at https://drive.google.com/file/d/1-nng3RxjHvUl5O6q8D8xYGoXXRdGpgKh/view?usp=drive_link
checkpoint_path = 'model_epoch_25.pth' 
checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=True)
model.load_state_dict(checkpoint)

test_dataset = DogHeartTestDataset("data/Test/Images", transforms=get_transform(512))
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

CSV_FILE = "test_vhs_scores_25.csv"

def save_vhs_to_csv(image_name, vhs_value):
    """Save image filename and VHS value in a CSV file."""
    file_exists = os.path.isfile(CSV_FILE)

    with open(CSV_FILE, mode="a", newline="") as file:
        writer = csv.writer(file)

        # Write header if file is new
        if not file_exists:
            writer.writerow(["Image Name", "VHS Value"])

        # Write image name and VHS value
        writer.writerow([image_name, vhs_value])

    print(f"✅ Saved: {image_name}, VHS: {vhs_value}")

with torch.no_grad():
        for data_batch in test_loader:
            if len(data_batch) == 4:
                image_names, images, points, vhs_gt = data_batch
            elif len(data_batch) == 2:
                images, image_names = data_batch
                points, vhs_gt = None, None  # No true points for unlabeled images
            else:
                print("Unexpected batch format")
                continue
            
            images = images.to(device)
            outputs = model(images)  # Generate predictions
            vhs_pred = calc_vhs(outputs)  # Compute VHS values
            
            # Convert tensors to numpy
            images_np = images.permute(0, 2, 3, 1).cpu().numpy()
            outputs_np = outputs.cpu().numpy()
            vhs_pred_np = vhs_pred.cpu().numpy()
            
            ## start : Loop through image names

            for i,name in enumerate(image_names):
                print(i,name)
                img = images_np[i]
                img = (img - img.min()) / (img.max() - img.min())  # Normalize for display
                pred_points = outputs_np[i].reshape(-1, 2) * img.shape[0]
                #print("pred points=>",len(pred_points))
                #print(pred_points)
                pred_vhs = vhs_pred_np[i].item()
                #print("vhs=>",pred_vhs)

                 # Save predictions in CSV format
                save_vhs_to_csv(name, pred_vhs)
                

            ## end : Loop through image names



NameError: name 'torch' is not defined

## Test Accuracy =>89.75

In [3]:
from graphviz import Digraph
import matplotlib.colors as mcolors

# Color palette (matplotlib 'tab10' + custom)
colors = {
    "stem": mcolors.to_hex("tab:blue"),
    "conv": mcolors.to_hex("tab:cyan"),
    "attention": mcolors.to_hex("tab:orange"),
    "se": mcolors.to_hex("tab:green"),
    "downsample": mcolors.to_hex("tab:red"),
    "regressor": mcolors.to_hex("tab:purple"),
}

def build_architecture():
    dot = Digraph(comment="MambaKeypointRegressor", format="png")
    dot.attr(rankdir="TB", splines="ortho", nodesep="0.5", ranksep="0.8")
    dot.attr("node", shape="box", style="rounded,filled", fontname="Helvetica", fontsize="10")

    # ---- Input ----
    dot.node("input", "Input Image\n[1,3,512,512]", shape="parallelogram", color="black")

    # ---- Stem ----
    with dot.subgraph(name="cluster_stem") as c:
        c.attr(label="MambaStem", color=colors["stem"], fontcolor=colors["stem"])
        c.node("stem_conv1", "Conv2d\n3→64, stride=2", color=colors["conv"])
        c.node("stem_norm1", "BatchNorm+SiLU", color=colors["conv"])
        c.node("stem_conv2", "Conv2d\n64→64", color=colors["conv"])
        c.node("stem_norm2", "BatchNorm+SiLU", color=colors["conv"])
        c.edges([("stem_conv1", "stem_norm1"), ("stem_norm1", "stem_conv2"), ("stem_conv2", "stem_norm2")])

    dot.edge("input", "stem_conv1")

    # ---- Stages ----
    stages = [
        {"name": "Stage1", "in_ch": 64, "out_ch": 128, "depth": 2, "attention": False},
        {"name": "Stage2", "in_ch": 128, "out_ch": 256, "depth": 3, "attention": True},
        {"name": "Stage3", "in_ch": 256, "out_ch": 512, "depth": 3, "attention": False},
        {"name": "Stage4", "in_ch": 512, "out_ch": 640, "depth": 3, "attention": True},
    ]

    prev_node = "stem_norm2"
    for stage in stages:
        with dot.subgraph(name=f"cluster_{stage['name']}") as c:
            c.attr(label=stage["name"], color=colors["stem"], fontcolor=colors["stem"])
            
            # Downsample
            c.node(f"{stage['name']}_down", 
                  f"Downsample\n{stage['in_ch']}→{stage['out_ch']}", 
                  color=colors["downsample"])
            dot.edge(prev_node, f"{stage['name']}_down")
            
            # Residual + Attention/SE Blocks
            for d in range(stage["depth"]):
                c.node(f"{stage['name']}_res{d}", 
                      f"ResidualBlock\n{stage['out_ch']}→{stage['out_ch']}", 
                      color=colors["conv"])
                c.node(f"{stage['name']}_att{d}", 
                      "AttentionMLP" if stage["attention"] else "SELayer", 
                      color=colors["attention"] if stage["attention"] else colors["se"])
                
                if d == 0:
                    c.edge(f"{stage['name']}_down", f"{stage['name']}_res{d}")
                else:
                    c.edge(f"{stage['name']}_att{d-1}", f"{stage['name']}_res{d}")
                
                c.edge(f"{stage['name']}_res{d}", f"{stage['name']}_att{d}")
            
            prev_node = f"{stage['name']}_att{stage['depth']-1}"

    # ---- Regressor ----
    with dot.subgraph(name="cluster_regressor") as c:
        c.attr(label="Regressor", color=colors["regressor"], fontcolor=colors["regressor"])
        c.node("pool", "GlobalAvgPool2d\n[1,640,1,1]", shape="ellipse")
        c.node("flatten", "Flatten\n→640", shape="ellipse")
        c.node("linear1", "Linear\n640→384")
        c.node("linear2", "Linear\n384→12")
        c.edges([("pool", "flatten"), ("flatten", "linear1"), ("linear1", "linear2")])

    dot.edge(prev_node, "pool")

    # ---- Output ----
    dot.node("output", "Keypoints\n[1,12]", shape="parallelogram", color="black")
    dot.edge("linear2", "output")

    return dot

# Generate and render
graph = build_architecture()
graph.render("mamba_architecture", view=True, cleanup=True)

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [2]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
